In [ ]:
import data.dataset as dataset

from models.model import HybridNet
from utils.anchors import Anchor
from utils.labels import decode_detection_output, get_detection_labels
from torch.utils.data import Subset
import torch
import torchvision
import numpy as np
import tqdm
import cv2
from loss.detection_loss import DetLoss
from loss.segmentation_loss import SegLoss
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from data.dataset import DetectionLabelDataset



In [ ]:
detection_class_num = 8
img_size = (375, 1242)
anchor_generator = Anchor(img_size)
detection_datset = DetectionLabelDataset(img_size, anchor_generator.anchors_list, get_detection_labels)
segment_dataset = dataset.SegmentDataset()


In [ ]:
img_transform = torchvision.transforms.Compose(
    [torchvision.transforms.Resize(512, interpolation=torchvision.transforms.InterpolationMode.NEAREST)]
)

def segmentation_rescale(X):
    img_batch = []
    label_batch = []
    for img, label in X:
        img = img_transform(img)
        label = img_transform(label)
        img_batch.append(img)
        label_batch.append(label)
    return torch.stack(img_batch,dim=0), torch.stack(label_batch,dim=0)

detection_dataloader = torch.utils.data.DataLoader(detection_datset, shuffle=True, batch_size=8)
segment_dataloader = torch.utils.data.DataLoader(segment_dataset, shuffle=True, batch_size=8, collate_fn = segmentation_rescale)

torch.save(detection_dataloader, "detection_dataset.pt")
torch.save(segment_dataloader, "segment_dataset.pt")




In [ ]:
# visualization detection label

anchors = list()
for anchor in anchor_generator.anchors_list[3:9]:
    anchor = anchor.reshape(-1,4)
    anchors.append(anchor)

anchors = torch.concatenate(anchors, 0)

batch_sample = next(iter(detection_dataloader))

fig, axes = plt.subplots(2,2, figsize=(24,8))
for i in range(4):
    sample_img = batch_sample[0][i]
    sample_box = batch_sample[1][i]
    sample_img = torch.permute(sample_img, (1,2,0))
    axes[i%2, i//2].imshow(sample_img)
    axes[i%2, i//2].axis('off')
    sample_cls = sample_box[:,0]
    object_ids = torch.where(sample_cls>0)
    sample_box = sample_box[:,1:]
    decoded_sample_box = decode_detection_output(anchors, sample_box)
    for ids in object_ids[0]:
        bbox = decoded_sample_box[ids]
        axes[i%2, i//2].add_patch(
            patches.Rectangle(
                (bbox[0] - bbox[2]/2, bbox[1] - bbox[3]/2),
                bbox[2], bbox[3],
                edgecolor='red',
                facecolor='none'
            )
        )


In [ ]:
net = HybridNet(128, anchor_generator.anchor_nums, detection_class_num)
net = net.to('cuda')

In [ ]:
# run segmentation training
import tqdm

def train_segmentation_task(net, segment_dataloader):
    loss = SegLoss()
    optimizer = torch.optim.Adam(net.parameters())
    pbar = tqdm.tqdm(total = len(segment_dataloader))
    acc_accum = 0
    acc_size = 0
    for batch in segment_dataloader:
        X, y = batch
        X, y = X.to('cuda'), y.to('cuda')
        y = torch.where(y==7, 1, 0)
        y = y.float()
        X = X / 255.0
        pred = net(X)
        pred_seg, pred_det= pred
        loss_score, acc = loss(pred_seg, y)
        optimizer.zero_grad()
        loss_score.backward()
        optimizer.step()
        acc_size += y.size()[0] * y.size()[1] * y.size()[2] * y.size()[3]
        acc_accum += acc.sum()
        acc = (acc_accum / acc_size).item()
        pbar.set_description(f"loss:{loss_score:.3f}, acc:{acc*100:.3f}%")
        pbar.update()
    pbar.close()


In [ ]:
# run detection training 

def train_detection_task(net, detection_dataloader):
    loss = DetLoss(detection_class_num)
    optimizer = torch.optim.Adam(net.parameters())

    pbar = tqdm.tqdm(total = len(detection_dataloader))
    for batch in detection_dataloader:
        X, y = batch
        X, y = X.to('cuda'), y.to('cuda')
        X = X / 255.0
        pred = net(X)
        pred_seg, pred_det = pred
        all_loss,cls_loss,box_loss = loss(pred_det, y)
        optimizer.zero_grad()
        all_loss.backward()
        optimizer.step()
        pbar.set_description(f"loss:{all_loss:.3f}, cls_loss{cls_loss:.3f}, box_loss{box_loss:.3f}")
        pbar.update()
    pbar.close()


In [ ]:
# multi task learning
for i in range(10):
        train_detection_task(net, detection_dataloader)
        train_segmentation_task(net, segment_dataloader)
        torch.save(net, f'trained_model_{i}.pth')